<a href="https://colab.research.google.com/github/minhthy1016/Credit-Innovators/blob/minhthy1016-patch-1/mailchimp_content_backfill_run_this.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install \
 apache-airflow==1.10.10 \
 --constraint \
        https://raw.githubusercontent.com/apache/airflow/1.10.10/requirements/requirements-python3.7.txt

In [ ]:
!pip install mailchimp3

     |████████████████████████████████| 102kB 4.8MB/s 


In [ ]:
import datetime
from airflow import DAG
from airflow.utils.dates import days_ago
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator
from airflow.contrib.operators import gcs_to_bq
import csv
import json
from pprint import pprint
# import subprocess
# import sys
# def install(package):
# subprocess.check_call([sys.executable, "-m", "pip3", "install", package])
# install('mailchimp3==3.0.14')
import time
from mailchimp3 import MailChimp
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup
import pandas as pd
import pandas_gbq
import itertools
from google.oauth2 import service_account
from google.auth import compute_engine
from google.cloud import storage
import os
from google.cloud import bigquery
from google.api_core.exceptions import BadRequest
from pprint import pprint
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/content/hhg-cdm-bcdeee60c0e1.json'

default_dag_args = {'ownder': 'data@hellohealthgroup,com',
                    'depend_on_past': False, 
                    'start_date': days_ago(1),
                    'retries': 2,
                    'project_id': 'hhg-cdm'}

with DAG(dag_id='mailchimp_backfill', description='mailchimp_backfill',
         default_args=default_dag_args) as dag:
    ####Dummy Variable For Start
    START = DummyOperator(task_id='START')
    ####Dummy Variable For End
    FINISH = DummyOperator(task_id='FINISH', depends_on_past=True)
    # logging.error('table name: '+table_name)

country_list = [
    {
        "country_code": "VN",
        "web_page": "Hello Bacsi"
    },
    {
        "country_code": "VN",
        "web_page": "Marry Baby-VN"
    },
    {
        "country_code": "MY",
        "web_page": "Hello Doktor"
    },
    {
        "country_code": "TH",
        "web_page": "Hello Khunmor"
    },
    {
        "country_code": "TW",
        "web_page": "Hello Yishi"
    },
    {
        "country_code": "PH",
        "web_page": "Hello Doctor"
    },
    {
        "country_code": "IN",
        "web_page": "Hello Swasthya"
    },
    {
        "country_code": "MM",
        "web_page": "Hello Sayarwon"
    },
    {
        "country_code": "KH",
        "web_page": "Hello Krupet"
    },
    {
        "country_code": "ID",
        "web_page": "Hello Sehat"
    }
]

country_code = {country['web_page']: country['country_code'] for country in country_list}
client = MailChimp(mc_api='1744313b5cc5b201df918cb4b9af764d-us11', mc_user='data@hellohealthgroup.com')

def html_parse_href(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([a.get('href') for a in soup.find_all('a', href=lambda x: 'http' in x and 'youtube' not in x)])
    except:
        return None

def html_parse_img(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([img.get('src') for img in soup.find_all('img')])
    except:
        return None

def html_parse_ytb(html_doc):
    try:
        soup = BeautifulSoup(html_doc, 'html.parser')
        return ', '.join([img.get('href') for img in soup.find_all('a', href=lambda x: "youtube" in x)])
    except:
        return None

def get_campaign():
    for i in range(10):
        try:
            campaigns = client.campaigns.all(get_all=True,
                                             fields="campaigns.id,campaigns.settings.subject_line,campaigns.settings.title,campaigns.settings.preview_text,campaigns.archive_url,campaigns.long_archive_url,campaigns.content_type,campaigns.create_time,campaigns.recipients.list_id,campaigns.recipients.list_name")
        except:
            continue
        break
    for campaign in campaigns['campaigns']:
        yield {
            'campaign_id': campaign.get('id'),
            'create_date': str(datetime.strptime(campaign.get('create_time'), '%Y-%m-%dT%H:%M:%S+00:00').date()),
            'create_time': str(campaign.get('create_time')),
            'subject_line': campaign.get('settings').get('subject_line'),
            'title': campaign.get('settings').get('title'),
            'list_id': campaign.get('recipients').get('list_id'),
            'list_name': campaign.get('recipients').get('list_name'),
            'content_type': campaign.get('content_type'),
            'archive_url': campaign.get('archive_url'),
            'long_archive_url': campaign.get('long_archive_url'),
            'country_code': country_code.get(campaign.get('recipients').get('list_name'))
        }

def get_content(campaigns):
    for campaign in campaigns[:10]:
        for i in range(10):
            try:
                content = client.campaigns.content.get(campaign_id=campaign['campaign_id'])
            #          print(content.get('plain_text'))
            except:
                continue
            break
        yield {
            'campaign_id': campaign['campaign_id'],
            'create_date': str(campaign['create_date']),
            'plain_text': content.get('plain_text'),
            'url': html_parse_href(content.get('html')),
            'img': html_parse_img(content.get('html')),
            'youtube': html_parse_ytb(content.get('html'))
        }

def rotate(l, n):
    return l[-n:] + l[:-n]

def streaming_to_bq(table_id,rows_to_insert,schema):
    #delete table
    client = bigquery.Client.from_service_account_json(r'/content/hhg-cdm-bcdeee60c0e1.json')

    client.delete_table(table_id, not_found_ok=True)  # Make an API request.
    print("Deleted table '{}'.".format(table_id))

    #create table
    table = bigquery.Table(table_id, schema=schema)
    table = client.create_table(table)  # Make an API request.
    print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))

    #streaming data
    errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
    if errors == []:
        print("New rows have been added for: ", table_id)
    else:
        print("Encountered errors while inserting rows: {}".format(errors))


def main():
    credentials = service_account.Credentials.from_service_account_file(
        r'/content/hhg-cdm-bcdeee60c0e1.json')
    all_campaigns = list(get_campaign())
    pprint(all_campaigns)
    
    all_content = list(get_content(all_campaigns))
    pprint(all_content)

    content_schema=[
    bigquery.SchemaField("campaign_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("create_date", "DATE", mode="NULLABLE"),
    bigquery.SchemaField("plain_text", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("url", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("img", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("youtube", "STRING", mode="NULLABLE"),
    ]
    #streaming_to_bq('hhg-ods.test.mailchimp_content', all_content, content_schema)
    streaming_to_bq('hhg-ods.hhg_lead_mailchimp.hhg_mailchimp_all_content_df', all_content, content_schema)

    campaigns_schema=[
    bigquery.SchemaField("campaign_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("create_date", "DATE", mode="NULLABLE"),
    bigquery.SchemaField("create_time", "TIMESTAMP", mode="NULLABLE"),
    bigquery.SchemaField("subject_line", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("title", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("list_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("list_name", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("content_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("archive_url", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("long_archive_url", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("country_code", "STRING", mode="NULLABLE"),
    ]
    streaming_to_bq('sandbox-247611.backup_test.mailchimp_campaign', all_campaigns, campaigns_schema)

main()

'''
run_hhg_mailchimp_all_content_di = PythonOperator(
    task_id='hhg_mailchimp_all_content_di',
    provide_context=False,
    depends_on_past=False,
    python_callable=streaming_to_bq,
    op_kwargs={'table_id': 'hhg-ods.test.content', 'rows_to_insert': all_content, 'schema': content_schema}
    dag=dag,
)
​
​
​
load_csv = gcs_to_bq.GoogleCloudStorageToBigQueryOperator(
    task_id='gcs_to_bq',
    bucket='hhg_mailchimp_campaign',
    source_objects=['all_campaigns_di.csv'],
    destination_project_dataset_table='hhg_lead_mailchimp.hhg_mailchimp_all_content_di',
    job_config = bigquery.LoadJobConfig(),
    job_config.allow_quoted_newlines = True,
    job_config.schema_fields=[
        {'name': 'campaign_id', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'create_date', 'type': 'DATE', 'mode': 'NULLABLE'},
        {'name': 'plain_text', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'url', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'title', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'img', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'youtube', 'type': 'STRING', 'mode': 'NULLABLE'},
​
    ],
    job_config.skip_leading_rows = 1,
    write_disposition='WRITE_TRUNCATE',
    dag=dag)
'''

# # Define DAG :
# #START >> run_hhg_mailchimp_all_content_di >> insert_hhg_mailchimp_df >> FINISH


Streaming output truncated to the last 5000 lines.
  'subject_line': 'Makan Pedas Sebelum Tidur Bisa Mengundang Mimpi Buruk, Kok '
                  'Bisa?',
  'title': 'HS Newsletter February 11th, 2019'},
 {'archive_url': 'http://eepurl.com/dEyjZH',
  'campaign_id': 'c0bf7baa55',
  'content_type': 'template',
  'country_code': 'VN',
  'create_date': '2018-08-20',
  'create_time': '2018-08-20T04:21:45+00:00',
  'list_id': '21913fa054',
  'list_name': 'Hello Bacsi',
  'long_archive_url': 'https://mailchi.mp/f33292f94001/bn-tin-sc-khe-s-4-bnh-au-c-dn-vn-phng',
  'subject_line': 'Bản tin sức khỏe số 4 - Bệnh đau cơ ở dân văn phòng',
  'title': 'Newsletter 20.08.2018'},
 {'archive_url': 'http://eepurl.com/hbF3Yn',
  'campaign_id': 'c0fb4246ae',
  'content_type': 'template',
  'country_code': 'MM',
  'create_date': '2020-08-25',
  'create_time': '2020-08-25T08:00:21+00:00',
  'list_id': '3d30d693ae',
  'list_name': 'Hello Sayarwon',
  'long_archive_url': 'https://us11.campaign-archive.com/

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

